In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
url= 'https://bit.ly/2Bkj2KT'

amazon= pd.read_csv(url, index_col=0)

In [3]:
amazon.shape

(999, 9)

In [4]:
amazon.head()

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0528881469,"[0, 0]",5.0,We got this GPS for my husband who is an (OTR)...,"06 2, 2013",AO94DHGC771SJ,amazdnu,Gotta have GPS!,1.370131e+09
1,0528881469,"[12, 15]",1.0,"I'm a professional OTR truck driver, and I bou...","11 25, 2010",AMO214LNFCEI4,Amazon Customer,Very Disappointed,1.290643e+09
2,0528881469,"[43, 45]",3.0,"Well, what can I say. I've had this unit in m...","09 9, 2010",A3N7T0DY83Y4IG,C. A. Freeman,1st impression,1.283990e+09
3,0528881469,"[9, 10]",2.0,"Not going to write a long review, even thought...","11 24, 2010",A1H8PY3QHMQQA0,"Dave M. Shaw ""mack dave""","Great grafics, POOR GPS",1.290557e+09
4,0528881469,"[0, 0]",1.0,I've had mine for a year and here's what we go...,"09 29, 2011",A24EV6RXELQZ63,Wayne Smith,"Major issues, only excuses for support",1.317254e+09


In [5]:
import nltk

In [6]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [7]:
docs= amazon['reviewText'].fillna('').str.lower().str.replace("[^a-z] ", "")

stopwords= nltk.corpus.stopwords.words('english')

def clean_sentence(text):
    words=text.split(' ')
    words_clean=[word for word in words if word not in stopwords]
    return ' '.join(words_clean)

docs_clean= docs.apply(clean_sentence)
docs_clean.head()

0    got gps husband (otrover road trucker impresse...
1    i'm professional otr truck driverand bought tn...
2    wellwhat say i've unit truck four days prior g...
3    going write long revieweven thought unit deser...
4    i've mine year here's gotit tries route non tr...
Name: reviewText, dtype: object

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer=CountVectorizer(min_df=10)
vectorizer.fit(docs_clean)
dtm_sparse=vectorizer.transform(docs_clean)

df_dtm= pd.DataFrame(dtm_sparse.toarray(), columns= vectorizer.get_feature_names())
df_dtm.head()

,10,100,1080p,16gb,32,32gb,34,34tv,3g,3inch,...,would,write,writing,written,wrong,year,years,yet,you,youtube
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,...,4,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,2,1,0,0,2,0,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0


In [10]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity([df_dtm['tablet'], df_dtm['touch']])

array([[1.        , 0.15939151],
       [0.15939151, 1.        ]])

In [11]:
cosine_similarity([df_dtm['nook'], df_dtm['book']])

array([[1.        , 0.59358784],
       [0.59358784, 1.        ]])

In [13]:
cs_matrix= cosine_similarity(df_dtm.T)
cs_matrix= pd.DataFrame(cs_matrix, columns=df_dtm.columns, index= df_dtm.columns)

cs_matrix['acer'].sort_values(ascending= False)

acer           1.000000
netbook        0.784538
cd             0.754328
windows        0.733895
dvd            0.722141
media          0.607948
player         0.606964
required       0.578315
upgraded       0.554700
performance    0.462250
etc            0.385543
additional     0.377426
external       0.340775
upgrade        0.326860
plug           0.313590
one            0.304543
playing        0.299948
usb            0.295545
box            0.292685
total          0.292353
checked        0.292353
mention        0.292353
picture        0.281970
junk           0.278747
figure         0.278747
just           0.262575
others         0.256410
rest           0.256410
watching       0.238705
plays          0.238705
                 ...   
now            0.000000
nstore         0.000000
pad            0.000000
number         0.000000
owned          0.000000
outside        0.000000
out            0.000000
otherwise      0.000000
other          0.000000
originally     0.000000
original       0

In [14]:
cs_matrix['camera'].sort_values(ascending= False)

camera        1.000000
pictures      0.277197
play          0.274717
files         0.258280
capable       0.225877
video         0.218348
test          0.215365
movie         0.202031
thinking      0.202031
sound         0.202031
smooth        0.198390
checked       0.191663
port          0.185195
hdmi          0.184684
now           0.182108
youtube       0.172292
length        0.171811
itit          0.171499
show          0.168359
tablets       0.167053
operating     0.164957
response      0.161985
though        0.158114
sit           0.158114
mei           0.151523
speed         0.149461
use           0.148652
itthe         0.147442
canon         0.147178
lots          0.144338
                ...   
local         0.000000
load          0.000000
name          0.000000
live          0.000000
listen        0.000000
list          0.000000
link          0.000000
line          0.000000
likely        0.000000
lights        0.000000
making        0.000000
manual        0.000000
mean       

In [15]:
cs_matrix['touch'].sort_values(ascending= False)

touch         1.000000
screen        0.633681
page          0.562060
kindle        0.514939
books         0.514019
interface     0.509097
nook          0.496120
book          0.484301
amazon        0.464486
documents     0.460401
reading       0.458610
read          0.457721
choose        0.447145
finger        0.431924
the           0.412532
turn          0.408728
back          0.406886
library       0.401498
buy           0.400344
use           0.396170
want          0.395059
new           0.383737
get           0.381836
go            0.379975
epub          0.376422
like          0.374309
kindlei       0.368037
would         0.367785
loaded        0.366083
simple        0.362520
                ...   
junk          0.000000
swing         0.000000
4inch         0.000000
42            0.000000
3inch         0.000000
required      0.000000
screws        0.000000
glad          0.000000
mounting      0.000000
hdtv          0.000000
hdmi          0.000000
hole          0.000000
tilt       